In [ ]:
import numpy as np
import tensorflow as tf
import gpflow as gpf
import matplotlib.pyplot as plt
from core.objectives import sample_GP_prior_utilities

In [ ]:
seed = 0
rng = np.random.default_rng(seed)
tf.random.set_seed(seed)
kernel = gpf.kernels.SquaredExponential(lengthscales=[0.1, 0.1])
lowers = [0., 0.]
uppers = [1., 1.]
num_agents = 2
num_actions = 8

In [ ]:
u = sample_GP_prior_utilities(num_agents,
                              kernel,
                              lowers,
                              uppers,
                              num_points=100,
                              rng=rng)

In [ ]:
from core.utils import combinations
from pathlib import Path

def plot_utilities_2d(u,
                      xlims,
                      ylims,
                      num_actions,
                      title="",
                      cmap="Spectral",
                      save=False,
                      save_dir="",
                      filename="",
                      show_plot=True):
    xmin, xmax = xlims
    ymin, ymax = ylims
    xdomain = np.linspace(xmin, xmax, num_actions)[:, None]
    ydomain = np.linspace(ymin, ymax, num_actions)[:, None]
    combs = combinations(xdomain, ydomain)
    xlabel = 'Agent 1 actions'
    ylabel = 'Agent 2 actions'

    u1_vals = u[0](combs)
    u1_reshaped = np.transpose(
        np.reshape(u1_vals, [num_actions, num_actions]))
    u2_vals = u[1](combs)
    u2_reshaped = np.transpose(
        np.reshape(u2_vals, [num_actions, num_actions]))

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(title, size=20)
    fig.set_size_inches(8, 4)
    fig.set_dpi(200)

    im1 = ax1.imshow(u1_reshaped,
                     interpolation='nearest',
                     extent=(xmin, xmax, ymin, ymax),
                     origin='lower',
                     cmap=cmap,
                     aspect=(xmax - xmin) / (ymax - ymin))
    # ax1.plot(inputs[:, 0], inputs[:, 1], 'ko', mew=2)
    # ax1.plot(*mean_stationary, marker='*', markersize=20, color='white')
    ax1.set_title("Agent 1 utility", size=16)
    ax1.set_xlabel(xlabel, size=12)
    ax1.set_ylabel(ylabel, size=12)
    fig.colorbar(im1, ax=ax1)

    im2 = ax2.imshow(u2_reshaped,
                     interpolation='nearest',
                     extent=(xmin, xmax, ymin, ymax),
                     origin='lower',
                     cmap=cmap,
                     aspect=(xmax - xmin) / (ymax - ymin))
    # ax2.plot(inputs[:, 0], inputs[:, 1], 'ko', mew=2)
    # ax2.plot(*f_std_argmin, marker='*', markersize=20, color='white')
    ax2.set_title("Agent 2 utility", size=16)
    ax2.set_xlabel(xlabel, size=12)
    ax2.set_ylabel(ylabel, size=12)
    fig.colorbar(im2, ax=ax2)
    fig.tight_layout()

    if save:
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        plt.savefig(save_dir + filename, bbox_inches='tight')

    if show_plot:
        plt.show()

In [ ]:
plot_utilities_2d(u,
                  (lowers[0], uppers[0]),
                  (lowers[1], uppers[1]),
                  num_actions)

In [ ]:
xmin, xmax = 0, 1
ymin, ymax = 0, 1
xdomain = np.linspace(xmin, xmax, num_actions)[:, None]
ydomain = np.linspace(ymin, ymax, num_actions)[:, None]
combs = combinations(xdomain, ydomain)
actions = np.squeeze(xdomain)

In [ ]:
def join_action(s,
                i,
                actions):
    """
    Given a strategy profile s, fixes the actions of all other agents, and replaces the action of the i-th agent
    with all possible actions.
    :param s: Array of shape (N, ). Strategy profile.
    :param i: int. Agent.
    :param actions: Array of shape (M, ). All possible actions.
    :return: Array of shape (M, N).
    """
    M = len(actions)
    joined = np.tile(s[None, :], (M, 1))
    joined[:, i] = actions
    return joined

In [ ]:
def best_response_payoff_pure(u,
                              S,
                              actions):
    """
    Calculates the best response payoff for each pure strategy profile in S, for each agent.
    :param u: List of utility functions.
    :param S: array of shape (M ** N, N). All possible pure strategy profiles of the N agents.
    :param actions: array of shape (M, ). All possible M actions.
    :return: array of shape (M ** N, N).
    """
    M = len(actions)
    _, N = S.shape
    brp = np.zeros((M ** N, N))

    for i in range(N):
        for j in range(len(S)):
            s = S[j]
            joined = join_action(s, i, actions)
            best_util = np.max(u[i](joined))
            current_util = u[i](s[None, :])
            brp[j, i] = best_util - current_util
    
    return brp

In [ ]:
brp = best_response_payoff_pure(u, combs, actions)

In [ ]:
def plot_brp_2d(brp,
                xlims,
                ylims,
                actions,
                title="",
                cmap="Spectral",
                save=False,
                save_dir="",
                filename="",
                show_plot=True):
    xmin, xmax = xlims
    ymin, ymax = ylims
    num_actions = len(actions)
    xlabel = 'Agent 1 actions'
    ylabel = 'Agent 2 actions'
    
    brp_max = np.max(brp, axis=1)
    brp_max_reshaped = np.transpose(
        np.reshape(brp_max, [num_actions, num_actions]))

    fig, (ax1) = plt.subplots(1, 1)
    fig.suptitle(title, size=12)
    fig.set_size_inches(6, 3)
    fig.set_dpi(200)

    im1 = ax1.imshow(brp_max_reshaped,
                     interpolation='nearest',
                     extent=(xmin, xmax, ymin, ymax),
                     origin='lower',
                     cmap=cmap,
                     aspect=(xmax - xmin) / (ymax - ymin))
    # ax1.plot(inputs[:, 0], inputs[:, 1], 'ko', mew=2)
    # ax1.plot(*mean_stationary, marker='*', markersize=20, color='white')
    ax1.set_title("Max best response payoff", size=12)
    ax1.set_xlabel(xlabel, size=8)
    ax1.set_ylabel(ylabel, size=8)
    fig.colorbar(im1, ax=ax1)
    
    fig.tight_layout()
    if save:
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        plt.savefig(save_dir + filename, bbox_inches='tight')
    if show_plot:
        plt.show()

In [ ]:
np.min(np.max(brp, axis=1))

In [ ]:
plot_brp_2d(brp, 
            (lowers[0], uppers[0]),   
            (lowers[1], uppers[1]),
            actions)